<a href="https://colab.research.google.com/github/hanghae-plus-AI/AI-1-hyeondata/blob/main/Chapter3_1_%EC%8B%AC%ED%99%94%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 목표

---

이번 과제는 자연어 task 중 하나인 MNLI를 해결하는 모델을 HuggingFace로 학습하는 것입니다. MNLI를 요약하면 다음과 같습니다.

- **입력**: premise에 해당하는 문장과 hypothesis에 해당하는 문장 두 개가 입력으로 들어옵니다.
- **출력:** 분류 문제로, 두 문장이 들어왔을 때 다음 세 가지를 예측하시면 됩니다.
    - **Entailment:** 두 문장에 논리적 모순이 없습니다.
    - **Neutral:** 두 문장은 논리적으로 관련이 없습니다.
    - **Contradiction:** 두 문장 사이에 논리적 모순이 존재합니다.

이 때, 다음 요구사항이 담긴 colab notebook을 만들어내시면 됩니다:

-  `load_dataset("nyu-mll/glue", "mnli")` 로 dataset을 불러옵니다.
    - 학습 때는 `train` split만 활용하셔야 합니다. 나머지 split은 사용불가입니다.
    - Validation data가 필요한 경우, `train` split에서 가져오셔야 합니다.
-  `trainer.train()`를 통해 학습된 log가 남아있어야 합니다.
-  Dataset의 `validation_matched`에 대한 성능을 출력하고, 50%를 넘기셔야 합니다.

# [4주차 심화과제] HuggingFace로 두 문장의 논리적 모순 분류하기
이번에는 HuggingFace로 두 문장의 논리적 모순을 분류 할 것입니다.
먼저 필요한 library들을 설치하고 import합시다.

In [1]:
!pip install transformers datasets evaluate accelerate scikit-learn

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00


In [2]:
import random
import evaluate
import numpy as np

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

## Dataset 준비

두 문장의 논리적 모순 분류하기 위해서 Dataset은 `nyu-mll/glue`을 활용합니다.

In [3]:
text_data = load_dataset("nyu-mll/glue", "mnli")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/52.2M [00:00<?, ?B/s]

(…)alidation_matched-00000-of-00001.parquet:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

(…)dation_mismatched-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

### train 데이터 구조 확인

In [4]:
text_data["train"][0]

{'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'hypothesis': 'Product and geography are what make cream skimming work. ',
 'label': 1,
 'idx': 0}

### train 데이터 레이블 개수 확인

In [5]:
np.unique(text_data["train"]["label"])

array([0, 1, 2])

보시다시피 각 data는 'premise' 해당하는 문장과 'hypothesis' 문장이 있으며tokenizer를 불러와서 미리 text들을 tokenize합니다.

Tokenizer를 실행할 때 넘겨주었던 `truncation` 옵션은 주어진 text가 일정 길이 이상이면 잘라내라는 의미입니다.
만약 특정 길이 값이 같이 주어지지 않는다면 `bert-base-cased`를 학습할 때 사용한 text의 최대 길이를 기준으로 값을 결정합니다.
(길이가 너무 긴 관계로 batcged = False 했습니다)

In [6]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def preprocess_function(data):
    return tokenizer(data["premise"],data["hypothesis"], truncation=True)

text_data_tokenized = text_data.map(preprocess_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

Map:   0%|          | 0/9796 [00:00<?, ? examples/s]

Map:   0%|          | 0/9847 [00:00<?, ? examples/s]

Tokenizer를 실행할 때 넘겨주었던 `truncation` 옵션은 주어진 text가 일정 길이 이상이면 잘라내라는 의미입니다.
만약 특정 길이 값이 같이 주어지지 않는다면 `bert-base-cased`를 학습할 때 사용한 text의 최대 길이를 기준으로 값을 결정합니다.

In [7]:
text_data_tokenized['train'][0].keys()

dict_keys(['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'])

마지막 출력 결과를 보면, `text`와 `label` 이외에 `input_ids`가 생기신 것을 확인하실 수 있습니다.
이는 우리가 `AutoTokenizer.from_pretrained`로 불러온 tokenizer로 text를 token들로 나누고 정수 index로 변환한 결과입니다.



In [8]:
text_data_tokenized

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9847
    })
})

이번에는 `train` data를 쪼개 training data와 validation data를 만들어보겠습니다.

In [9]:
text_data_split = text_data_tokenized['train'].train_test_split(test_size=0.2)
text_data_train, text_data_val = text_data_split['train'], text_data_split['test']
text_data_test = text_data_tokenized['validation_matched']

HuggingFace `datasets`로 불러온 dataset은 `train_test_split`으로 쉽게 쪼갤 수 있습니다.

다음은 각 split의 크기입니다.

In [10]:
len(text_data_train), len(text_data_val), len(text_data_test)

(314161, 78541, 9815)

## Model 구현

이번에는 text 분류를 수행할 Transformer를 구현합니다.
이전에는 Transformer의 구성 요소들을 직접 구현하여 합쳤습니다.
이번에는 HuggingFace의 BERT를 활용하여 인자만 넘겨주는 식으로 구현해보겠습니다:

In [11]:
from transformers import BertConfig

config = BertConfig()

config.hidden_size = 64  # BERT layer의 기본 hidden dimension
config.intermediate_size = 64  # FFN layer의 중간 hidden dimension
config.num_hidden_layers = 2  # BERT layer의 개수
config.num_attention_heads = 4  # Multi-head attention에서 사용하는 head 개수
config.num_labels = 4  # 마지막에 예측해야 하는 분류 문제의 class 개수

model = AutoModelForSequenceClassification.from_config(config)

BERT는 이전에 배운 Transformer의 architecture를 그대로 사용합니다.
그래서 BERT의 옵션들만 수정하면 vanilla Transformer를 쉽게 구현할 수 있습니다.

Transformer 구현 이외에 분류 문제에 맞춰 첫 번째 token을 linear classifier를 거치는 등의 과정은 `AutoModelForSequenceClassification`이 구현해줍니다.
즉, 우리가 `config`로 넘겨주는 BERT의 마지막에 linear classifier를 달아주는 역할을 합니다.

## 학습 코드

다음은 위에서 구현한 Transformer를 nyu-mll/glue 학습하는 코드를 구현합니다.
먼저 다음과 같이 학습 인자들을 정의합니다.

In [12]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='hf_transformer',  # 모델, log 등을 저장할 directory
    num_train_epochs=10,  # epoch 수
    per_device_train_batch_size=128,  # training data의 batch size
    per_device_eval_batch_size=128,  # validation data의 batch size
    logging_strategy="epoch",  # Epoch가 끝날 때마다 training loss 등을 log하라는 의미
    do_train=True,  # 학습을 진행하겠다는 의미
    do_eval=True,  # 학습 중간에 validation data에 대한 평가를 수행하겠다는 의미
    eval_strategy="epoch",  # 매 epoch가 끝날 때마다 validation data에 대한 평가를 수행한다는 의미
    save_strategy="epoch",  # 매 epoch가 끝날 때마다 모델을 저장하겠다는 의미
    learning_rate=1e-3,  # optimizer에 사용할 learning rate
    load_best_model_at_end=True  # 학습이 끝난 후, validation data에 대한 성능이 가장 좋은 모델을 채택하겠다는 의미
)

각각의 부분들은 이전 주차에서 배웠던 내용들을 설정하는 것에 불과하다는 것을 알 수 있습니다.
요약하면 다음과 같습니다:
- `epochs`: training data를 몇 번 반복할 것인지 결정합니다.
- `batch_size`: training data를 얼마나 잘게 잘라서 학습할 것인지 결정합니다.
- `learning_rate`: optimizer의 learning rate를 얼마로 할 것인지 결정합니다.
위의 부분들 이외에도 사소한 구현 요소들도 지정할 수 있습니다.

다음은 loss 이외의 평가 함수들을 구현하는 방법입니다.

In [13]:
import evaluate

accuracy = evaluate.load("accuracy")


def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

모델, training 인자, training과 validation data, 부가적인 평가 함수, 그리고 tokenizer를 넘겨주면 끝입니다.
별개로 early stopping과 기능도 적용

아래와 같이 만든 `Trainer`는 다음과 같이 학습을 할 수 있습니다.

In [14]:
from transformers import EarlyStoppingCallback


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=text_data_train,
    eval_dataset=text_data_val,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=1)]
)

In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.981300,0.911623,0.575215
2,0.882500,0.880343,0.592340
3,0.824000,0.860279,0.612992
4,0.776000,0.858238,0.619867
5,0.731200,0.869062,0.618671


TrainOutput(global_step=12275, training_loss=0.8390143083312118, metrics={'train_runtime': 469.9854, 'train_samples_per_second': 6684.484, 'train_steps_per_second': 52.236, 'total_flos': 65694524282040.0, 'train_loss': 0.8390143083312118, 'epoch': 5.0})

보시다시피 training loss는 잘 떨어지는 반면, validation loss는 중간부터 올라가는 것을 볼 수 있습니다.
Overfitting이 일어났다고 볼 수 있습니다.

위와 같이 학습이 끝난 후 validation loss가 가장 낮은 모델을 가지고 test data의 성능을 평가하는 것은 다음과 같이 구현할 수 있습니다.

In [16]:
trainer.evaluate(text_data_test)

{'eval_loss': 0.8299634456634521,
 'eval_accuracy': 0.6418746816097809,
 'eval_runtime': 1.7306,
 'eval_samples_per_second': 5671.295,
 'eval_steps_per_second': 44.492,
 'epoch': 5.0}

이전에 학습 인자에서 `load_best_model_at_end=True`를 넘겨줬기 때문에 `trainer`는 학습이 끝난 후, 기본적으로 validation loss가 가장 좋은 모델을 가지고 `evaluate`를 진행합니다.
실제로 결과를 보면 `eval_accuracy`가 0.64도로 64% 정확도로 보이고 있습니다.

평가할 때 사용한 모델은 다음과 같이 저장할 수 있습니다.

In [17]:
trainer.save_model()

그리고 저장한 모델을 가지고 다른 예시들을 예측하는 것은 다음과 같이 구현할 수 있습니다.

In [18]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="./hf_transformer/", device='cuda')

for i in range(10):
    # 토큰화된 입력 가져오기
    input_ids = text_data_tokenized['validation_matched'][i]['input_ids']

    # 토큰화된 입력을 다시 텍스트로 변환
    decoded_text = tokenizer.decode(input_ids, skip_special_tokens=False)

    print(f"Input: {decoded_text}")

    # 분류기에 입력
    result = classifier(decoded_text)

    print(f"Sentiment: {result[0]['label']}, Score: {result[0]['score']:.4f}\n")

Input: [CLS] The new rights are nice enough [SEP] Everyone really likes the newest benefits [SEP]
Sentiment: LABEL_2, Score: 0.5745

Input: [CLS] This site includes a list of all award winners and a searchable database of Government Executive articles. [SEP] The Government Executive articles housed on the website are not able to be searched. [SEP]
Sentiment: LABEL_2, Score: 0.7109

Input: [CLS] uh i don't know i i have mixed emotions about him uh sometimes i like him but at the same times i love to see somebody beat him [SEP] I like him for the most part, but would still enjoy seeing someone beat him. [SEP]
Sentiment: LABEL_2, Score: 0.4771

Input: [CLS] yeah i i think my favorite restaurant is always been the one closest you know the closest as long as it's it meets the minimum criteria you know of good food [SEP] My favorite restaurants are always at least a hundred miles away from my house. [SEP]
Sentiment: LABEL_2, Score: 0.6615

Input: [CLS] i don't know um do you do a lot of camp

HuggingFace의 `pipeline`은 다양한 모델들에 대하여 서비스에 사용할 수 있는 형태들을 제공합니다.
여기서는 'validation_matched'데이터 에서 input_ids를 입력으로, label이 0(두 문장이 논리적 모순이 없는지)인지 1(두 문장이 논리적으로 관련이 없는지)인지 2(두 문장 사이에 논리적 모순이 존재하는지)를 예측 결과를 보여줄 뿐만 아니라 그 신뢰도를 `score`로 넘겨주게 됩니다.

이처럼 HuggingFace를 활용하면 모델이나 예측, 학습 코드를 구현할 필요 없이 인자로 설정값들만 넘겨주면 쉽게 구현 할 수 있습니다.

추가로 validation_matched의 10개의 데이터를 예측한 모습입니다!

## Transfer learning

이번에는 task는 영화 리뷰 감정 분석으로 유지하되, 모델을 distilbert를 fine-tuning하는 것으로 바꿔보겠습니다.
모델은 다음과 같이 불러올 수 있습니다.

In [19]:
id2label = {
    0: "entailment",
    1: "neutral",
    2: "contradiction"
    }
label2id = {"entailment": 0, "neutral": 1,"contradiction":2}

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


`id2label`과 `label2id`는 예측 결과의 가독성을 위해서 추가하였습니다.
기존과 같이 `AutoModelForSequenceClassification`를 사용하고 있습니다.
달라진 점은 `from_config`가 아닌 `from_pretrained`를 사용한다는 것입니다.
`from_pretrained`를 사용하면 HuggingFace hub에 있는 pre-trained 모델들을 사용할 수 있습니다.
감정 분석 문제의 class 수에 맞춰 `num_labels`를 3로 설정하면 모델 구현은 거의 끝났습니다.

이번에는 마지막 layer를 제외한 parameter들을 freeze해보겠습니다.
그 전에 freeze 해야 하는 layer들을 확인하기 위해 `model`을 출력해보겠습니다.

In [20]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

보시다시피 `distilbert`는 기존의 distilbert 모델에 해당하고 나머지 `pre_classifier`, `classifier`는 text 분류를 위해 새롭게 추가된 layer들입니다.
즉, 다음과 같이 `distilbert`에 해당하는 parameter들만 freeze하면 됩니다.

In [21]:
for param in model.distilbert.parameters():
  param.requires_grad = False

이러면 모델 구현은 완전히 마쳤습니다.
다음은 distilbert를 pre-train할 때 사용했던 tokenizer를 불러오고, 이 tokenizer를 가지고 이전과 똑같이 nyu-mll/glue dataset를 전처리합니다.

In [22]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

def preprocess_function(data):
    return tokenizer(data["premise"],data["hypothesis"], truncation=True)

text_data_tokenized = text_data.map(preprocess_function, batched=True)
text_data_split = text_data_tokenized['train'].train_test_split(test_size=0.2)
text_data_train, text_data_val = text_data_split['train'], text_data_split['test']
text_data_test = text_data_tokenized['validation_matched']

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

Map:   0%|          | 0/9796 [00:00<?, ? examples/s]

Map:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [23]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='hf_distilbert',  # 모델, log 등을 저장할 directory
    num_train_epochs=3,  # epoch 수 (학습시간이 너무 긴 이유로 epoch를 줄임)
    per_device_train_batch_size=128,  # training data의 batch size
    per_device_eval_batch_size=128,  # validation data의 batch size
    logging_strategy="epoch",  # Epoch가 끝날 때마다 training loss 등을 log하라는 의미
    do_train=True,  # 학습을 진행하겠다는 의미
    do_eval=True,  # 학습 중간에 validation data에 대한 평가를 수행하겠다는 의미
    eval_strategy="epoch",  # 매 epoch가 끝날 때마다 validation data에 대한 평가를 수행한다는 의미
    save_strategy="epoch",  # 매 epoch가 끝날 때마다 모델을 저장하겠다는 의미
    learning_rate=1e-3,  # optimizer에 사용할 learning rate
    load_best_model_at_end=True  # 학습이 끝난 후, validation data에 대한 성능이 가장 좋은 모델을 채택하겠다는 의미
)

나머지는 기존의 학습과정과 완전히 동일합니다. `training_args`는 기존과 똑같이 활용할 때 distilbert를 fine-tuning하는 코드는 다음과 같습니다.

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=text_data_train,
    eval_dataset=text_data_val,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

trainer.train()
trainer.save_model()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.020500,0.985904,0.515209
2,0.994600,0.975299,0.522402
3,0.983600,0.966052,0.526655


학습한 모델에 대한 결과는 다음과 같습니다.

In [26]:
trainer.predict(text_data_test)

PredictionOutput(predictions=array([[-0.06267764,  1.0460598 , -1.3684157 ],
       [-1.0205168 , -0.67920333,  1.6721483 ],
       [ 0.1156906 , -0.0259801 , -0.35647166],
       ...,
       [ 1.186481  , -0.11220684, -1.1196717 ],
       [ 0.6760238 , -0.20928055, -0.55917287],
       [-0.37670708, -0.23817283,  0.5958774 ]], dtype=float32), label_ids=array([1, 2, 0, ..., 0, 0, 2]), metrics={'test_loss': 0.9626114368438721, 'test_accuracy': 0.5313295975547632, 'test_runtime': 8.0611, 'test_samples_per_second': 1217.568, 'test_steps_per_second': 9.552})

In [25]:
classifier = pipeline("sentiment-analysis", model="./hf_distilbert/", device='cuda')

for i in range(10):
    # 토큰화된 입력 가져오기
    input_ids = text_data_tokenized['validation_matched'][i]['input_ids']

    # 토큰화된 입력을 다시 텍스트로 변환
    decoded_text = tokenizer.decode(input_ids, skip_special_tokens=False)

    print(f"Input: {decoded_text}")

    # 분류기에 입력
    result = classifier(decoded_text)

    print(f"Sentiment: {result[0]['label']}, Score: {result[0]['score']:.4f}\n")

Input: [CLS] the new rights are nice enough [SEP] everyone really likes the newest benefits [SEP]
Sentiment: neutral, Score: 0.6006

Input: [CLS] this site includes a list of all award winners and a searchable database of government executive articles. [SEP] the government executive articles housed on the website are not able to be searched. [SEP]
Sentiment: contradiction, Score: 0.8318

Input: [CLS] uh i don't know i i have mixed emotions about him uh sometimes i like him but at the same times i love to see somebody beat him [SEP] i like him for the most part, but would still enjoy seeing someone beat him. [SEP]
Sentiment: neutral, Score: 0.4215

Input: [CLS] yeah i i think my favorite restaurant is always been the one closest you know the closest as long as it's it meets the minimum criteria you know of good food [SEP] my favorite restaurants are always at least a hundred miles away from my house. [SEP]
Sentiment: neutral, Score: 0.5997

Input: [CLS] i don't know um do you do a lot o

### Transfer learning을 했을 경우 학습 횟수를 줄인 이유일 수도 있지만 정확도가 더 떨어진 것을 볼 수 있다.

# input에는 한 줄 밖에 넣을 수 없기에 실험삼아서 'premise'와 'hypothesis'를 가운데 "+"를 넣어서 한 줄로 만들어서 분류를 해보고자 했습니다.

Tokenizer를 실행할 때 넘겨주었던 `truncation` 옵션은 주어진 text가 일정 길이 이상이면 잘라내라는 의미입니다.
만약 특정 길이 값이 같이 주어지지 않는다면 `bert-base-cased`를 학습할 때 사용한 text의 최대 길이를 기준으로 값을 결정합니다.
(길이가 너무 긴 관계로 batcged = False 했습니다)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def preprocess_function(data):
    return tokenizer(str(data["premise"]) + "+"+str(data["hypothesis"]), truncation=True)

text_data_tokenized = text_data.map(preprocess_function, batched=False)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/9796 [00:00<?, ? examples/s]

Tokenizer를 실행할 때 넘겨주었던 `truncation` 옵션은 주어진 text가 일정 길이 이상이면 잘라내라는 의미입니다.
만약 특정 길이 값이 같이 주어지지 않는다면 `bert-base-cased`를 학습할 때 사용한 text의 최대 길이를 기준으로 값을 결정합니다.

In [ ]:
text_data_tokenized['train'][0].keys()

dict_keys(['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'])

마지막 출력 결과를 보면, `text`와 `label` 이외에 `input_ids`가 생기신 것을 확인하실 수 있습니다.
이는 우리가 `AutoTokenizer.from_pretrained`로 불러온 tokenizer로 text를 token들로 나누고 정수 index로 변환한 결과입니다.



In [ ]:
text_data_tokenized

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9847
    })
})

이번에는 `train` data를 쪼개 training data와 validation data를 만들어보겠습니다.

In [ ]:
text_data_split = text_data_tokenized['train'].train_test_split(test_size=0.2)
text_data_train, text_data_val = text_data_split['train'], text_data_split['test']
text_data_test = text_data_tokenized['validation_matched']

HuggingFace `datasets`로 불러온 dataset은 `train_test_split`으로 쉽게 쪼갤 수 있습니다.

다음은 각 split의 크기입니다.

In [ ]:
len(text_data_train), len(text_data_val), len(text_data_test)

(314161, 78541, 9815)

## Model 구현

이번에는 text 분류를 수행할 Transformer를 구현합니다.
이전에는 Transformer의 구성 요소들을 직접 구현하여 합쳤습니다.
이번에는 HuggingFace의 BERT를 활용하여 인자만 넘겨주는 식으로 구현해보겠습니다:

In [ ]:
from transformers import BertConfig

config = BertConfig()

config.hidden_size = 64  # BERT layer의 기본 hidden dimension
config.intermediate_size = 64  # FFN layer의 중간 hidden dimension
config.num_hidden_layers = 2  # BERT layer의 개수
config.num_attention_heads = 4  # Multi-head attention에서 사용하는 head 개수
config.num_labels = 4  # 마지막에 예측해야 하는 분류 문제의 class 개수

model = AutoModelForSequenceClassification.from_config(config)

BERT는 이전에 배운 Transformer의 architecture를 그대로 사용합니다.
그래서 BERT의 옵션들만 수정하면 vanilla Transformer를 쉽게 구현할 수 있습니다.

Transformer 구현 이외에 분류 문제에 맞춰 첫 번째 token을 linear classifier를 거치는 등의 과정은 `AutoModelForSequenceClassification`이 구현해줍니다.
즉, 우리가 `config`로 넘겨주는 BERT의 마지막에 linear classifier를 달아주는 역할을 합니다.

## 학습 코드

다음은 위에서 구현한 Transformer를 nyu-mll/glue 학습하는 코드를 구현합니다.
먼저 다음과 같이 학습 인자들을 정의합니다.

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='hf_transformer',  # 모델, log 등을 저장할 directory
    num_train_epochs=10,  # epoch 수
    per_device_train_batch_size=128,  # training data의 batch size
    per_device_eval_batch_size=128,  # validation data의 batch size
    logging_strategy="epoch",  # Epoch가 끝날 때마다 training loss 등을 log하라는 의미
    do_train=True,  # 학습을 진행하겠다는 의미
    do_eval=True,  # 학습 중간에 validation data에 대한 평가를 수행하겠다는 의미
    eval_strategy="epoch",  # 매 epoch가 끝날 때마다 validation data에 대한 평가를 수행한다는 의미
    save_strategy="epoch",  # 매 epoch가 끝날 때마다 모델을 저장하겠다는 의미
    learning_rate=1e-3,  # optimizer에 사용할 learning rate
    load_best_model_at_end=True  # 학습이 끝난 후, validation data에 대한 성능이 가장 좋은 모델을 채택하겠다는 의미
)

각각의 부분들은 이전 주차에서 배웠던 내용들을 설정하는 것에 불과하다는 것을 알 수 있습니다.
요약하면 다음과 같습니다:
- `epochs`: training data를 몇 번 반복할 것인지 결정합니다.
- `batch_size`: training data를 얼마나 잘게 잘라서 학습할 것인지 결정합니다.
- `learning_rate`: optimizer의 learning rate를 얼마로 할 것인지 결정합니다.
위의 부분들 이외에도 사소한 구현 요소들도 지정할 수 있습니다.

다음은 loss 이외의 평가 함수들을 구현하는 방법입니다.

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")


def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

모델, training 인자, training과 validation data, 부가적인 평가 함수, 그리고 tokenizer를 넘겨주면 끝입니다.
별개로 early stopping과 기능도 적용

아래와 같이 만든 `Trainer`는 다음과 같이 학습을 할 수 있습니다.

In [ ]:
from transformers import EarlyStoppingCallback


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=text_data_train,
    eval_dataset=text_data_val,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=1)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.015300,0.963853,0.522835
2,0.944800,0.941308,0.541195
3,0.903200,0.926983,0.549751
4,0.867000,0.942226,0.549458


TrainOutput(global_step=9820, training_loss=0.9325868563836558, metrics={'train_runtime': 378.905, 'train_samples_per_second': 8291.287, 'train_steps_per_second': 64.792, 'total_flos': 52752265050120.0, 'train_loss': 0.9325868563836558, 'epoch': 4.0})

보시다시피 training loss는 잘 떨어지는 반면, validation loss는 중간부터 올라가는 것을 볼 수 있습니다.
Overfitting이 일어났다고 볼 수 있습니다.

위와 같이 학습이 끝난 후 validation loss가 가장 낮은 모델을 가지고 test data의 성능을 평가하는 것은 다음과 같이 구현할 수 있습니다.

In [ ]:
trainer.evaluate(text_data_test)

{'eval_loss': 0.9131302237510681,
 'eval_accuracy': 0.5685175751400917,
 'eval_runtime': 1.7447,
 'eval_samples_per_second': 5625.631,
 'eval_steps_per_second': 44.134,
 'epoch': 4.0}

이전에 학습 인자에서 `load_best_model_at_end=True`를 넘겨줬기 때문에 `trainer`는 학습이 끝난 후, 기본적으로 validation loss가 가장 좋은 모델을 가지고 `evaluate`를 진행합니다.
실제로 결과를 보면 `eval_loss`가 0.56정도로 보이고 있습니다.

평가할 때 사용한 모델은 다음과 같이 저장할 수 있습니다.

In [ ]:
trainer.save_model()

그리고 저장한 모델을 가지고 다른 예시들을 예측하는 것은 다음과 같이 구현할 수 있습니다.

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="./hf_transformer/", device='cuda')

for i in range(10):
  print(text_data_tokenized['validation_matched'][i]['premise'] + "+" + text_data_tokenized['validation_matched'][i]['hypothesis'])
  print(classifier(text_data_tokenized['validation_matched'][i]['premise'] + "+" + text_data_tokenized['validation_matched'][i]['hypothesis']))


The new rights are nice enough+Everyone really likes the newest benefits 
[{'label': 'LABEL_1', 'score': 0.5555612444877625}]
This site includes a list of all award winners and a searchable database of Government Executive articles.+The Government Executive articles housed on the website are not able to be searched.
[{'label': 'LABEL_2', 'score': 0.9319422841072083}]
uh i don't know i i have mixed emotions about him uh sometimes i like him but at the same times i love to see somebody beat him+I like him for the most part, but would still enjoy seeing someone beat him.
[{'label': 'LABEL_1', 'score': 0.6511645317077637}]
yeah i i think my favorite restaurant is always been the one closest  you know the closest as long as it's it meets the minimum criteria you know of good food+My favorite restaurants are always at least a hundred miles away from my house. 
[{'label': 'LABEL_1', 'score': 0.4312286376953125}]
i don't know um do you do a lot of camping+I know exactly.
[{'label': 'LABEL_2', 

HuggingFace의 `pipeline`은 다양한 모델들에 대하여 서비스에 사용할 수 있는 형태들을 제공합니다.
여기서는 문장 'premise'와 'validation_matched'를 "+"를 입력으로, label이 0(두 문장이 논리적 모순이 없는지)인지 1(두 문장이 논리적으로 관련이 없는지)인지 2(두 문장 사이에 논리적 모순이 존재하는지)를 예측 결과를 보여줄 뿐만 아니라 그 신뢰도를 `score`로 넘겨주게 됩니다.

이처럼 HuggingFace를 활용하면 모델이나 예측, 학습 코드를 구현할 필요 없이 인자로 설정값들만 넘겨주면 쉽게 구현 할 수 있습니다.

추가로 validation_matched의 10개의 데이터를 예측한 모습입니다!

## Transfer learning

이번에는 task는 영화 리뷰 감정 분석으로 유지하되, 모델을 distilbert를 fine-tuning하는 것으로 바꿔보겠습니다.
모델은 다음과 같이 불러올 수 있습니다.

In [ ]:
id2label = {
    0: "entailment",
    1: "neutral",
    2: "contradiction"
    }
label2id = {"entailment": 0, "neutral": 1,"contradiction":2}

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


`id2label`과 `label2id`는 예측 결과의 가독성을 위해서 추가하였습니다.
기존과 같이 `AutoModelForSequenceClassification`를 사용하고 있습니다.
달라진 점은 `from_config`가 아닌 `from_pretrained`를 사용한다는 것입니다.
`from_pretrained`를 사용하면 HuggingFace hub에 있는 pre-trained 모델들을 사용할 수 있습니다.
감정 분석 문제의 class 수에 맞춰 `num_labels`를 3로 설정하면 모델 구현은 거의 끝났습니다.

이번에는 마지막 layer를 제외한 parameter들을 freeze해보겠습니다.
그 전에 freeze 해야 하는 layer들을 확인하기 위해 `model`을 출력해보겠습니다.

In [ ]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

보시다시피 `distilbert`는 기존의 distilbert 모델에 해당하고 나머지 `pre_classifier`, `classifier`는 text 분류를 위해 새롭게 추가된 layer들입니다.
즉, 다음과 같이 `distilbert`에 해당하는 parameter들만 freeze하면 됩니다.

In [ ]:
for param in model.distilbert.parameters():
  param.requires_grad = False

이러면 모델 구현은 완전히 마쳤습니다.
다음은 distilbert를 pre-train할 때 사용했던 tokenizer를 불러오고, 이 tokenizer를 가지고 이전과 똑같이 nyu-mll/glue dataset를 전처리합니다.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

def preprocess_function(data):
    return tokenizer(str(data["premise"]) + "+"+str(data["hypothesis"]), truncation=True)

text_data_tokenized = text_data.map(preprocess_function, batched=False)
text_data_split = text_data_tokenized['train'].train_test_split(test_size=0.2)
text_data_train, text_data_val = text_data_split['train'], text_data_split['test']
text_data_test = text_data_tokenized['validation_matched']

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

Map:   0%|          | 0/9796 [00:00<?, ? examples/s]

Map:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='hf_distilbert',  # 모델, log 등을 저장할 directory
    num_train_epochs=10,  # epoch 수
    per_device_train_batch_size=128,  # training data의 batch size
    per_device_eval_batch_size=128,  # validation data의 batch size
    logging_strategy="epoch",  # Epoch가 끝날 때마다 training loss 등을 log하라는 의미
    do_train=True,  # 학습을 진행하겠다는 의미
    do_eval=True,  # 학습 중간에 validation data에 대한 평가를 수행하겠다는 의미
    eval_strategy="epoch",  # 매 epoch가 끝날 때마다 validation data에 대한 평가를 수행한다는 의미
    save_strategy="epoch",  # 매 epoch가 끝날 때마다 모델을 저장하겠다는 의미
    learning_rate=1e-3,  # optimizer에 사용할 learning rate
    load_best_model_at_end=True  # 학습이 끝난 후, validation data에 대한 성능이 가장 좋은 모델을 채택하겠다는 의미
)

나머지는 기존의 학습과정과 완전히 동일합니다. `training_args`는 기존과 똑같이 활용할 때 distilbert를 fine-tuning하는 코드는 다음과 같습니다.

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=text_data_train,
    eval_dataset=text_data_val,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

trainer.train()
trainer.save_model()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.000000,0.958825,0.535886
2,0.973400,0.958811,0.536955
3,0.965200,0.938294,0.550171
4,0.957100,0.932876,0.555850
5,0.951700,0.928737,0.557823
6,0.945800,0.924551,0.560612
7,0.941400,0.918258,0.566328
8,0.937500,0.918535,0.565921
9,0.933400,0.915485,0.567602
10,0.928900,0.912244,0.568888


학습한 모델에 대한 결과는 다음과 같습니다.

In [ ]:
trainer.predict(text_data_test)

PredictionOutput(predictions=array([[-0.3335847 ,  1.1757344 , -1.5620683 ],
       [-1.579877  , -1.0748188 ,  2.293501  ],
       [ 0.44951993,  0.03506072, -0.7433631 ],
       ...,
       [ 1.0674206 ,  0.00622229, -1.3473322 ],
       [ 0.99848443, -0.6602617 , -0.86747396],
       [-0.74312574, -0.6515769 ,  1.0988998 ]], dtype=float32), label_ids=array([1, 2, 0, ..., 0, 0, 2]), metrics={'test_loss': 0.9110174179077148, 'test_accuracy': 0.5687213448802853, 'test_runtime': 8.0556, 'test_samples_per_second': 1218.412, 'test_steps_per_second': 9.559})

In [ ]:
classifier = pipeline("sentiment-analysis", model="./hf_distilbert/", device='cuda')

for i in range(10):
  print(text_data_tokenized['validation_matched'][i]['premise'] + "+" + text_data_tokenized['validation_matched'][i]['hypothesis'])
  print(classifier(text_data_tokenized['validation_matched'][i]['premise'] + "+" + text_data_tokenized['validation_matched'][i]['hypothesis']))

The new rights are nice enough+Everyone really likes the newest benefits 
[{'label': 'neutral', 'score': 0.7777426242828369}]
This site includes a list of all award winners and a searchable database of Government Executive articles.+The Government Executive articles housed on the website are not able to be searched.
[{'label': 'contradiction', 'score': 0.9476558566093445}]
uh i don't know i i have mixed emotions about him uh sometimes i like him but at the same times i love to see somebody beat him+I like him for the most part, but would still enjoy seeing someone beat him.
[{'label': 'entailment', 'score': 0.5091540813446045}]
yeah i i think my favorite restaurant is always been the one closest  you know the closest as long as it's it meets the minimum criteria you know of good food+My favorite restaurants are always at least a hundred miles away from my house. 
[{'label': 'neutral', 'score': 0.5312472581863403}]
i don't know um do you do a lot of camping+I know exactly.
[{'label': 'c

## 결론

"+" 를 넣는것 보다 `[SEP]` 토큰을 활용해서 학습하는 것이 성능이 더 좋은것을 보이고 있다.

# 추가로 sequence-to-sequence 모델 구조와 프롬프트를 이용한 분류
## (현재 최신 모델중 하나인 Llama-3.2-3B를 이용 )
### 시간 관계상 ... 실패.. 오류나는 부분해서 추후 시도 예정

In [27]:
import random
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorWithPadding


In [29]:
!pip install huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [30]:
# Dataset 준비
text_data = load_dataset("nyu-mll/glue", "mnli")

# 모델 및 토크나이저 초기화
model_name = "meta-llama/Llama-3.2-3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Llama 모델은 기본적으로 패딩 토큰이 없으므로, 추가해줍니다.
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [31]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

prefix = """
Task: Classify the logical relationship between two given sentences (premise and hypothesis).
Output: Predict one of the following three categories:
- Entailment: There is no logical contradiction between the two sentences.
- Neutral: The two sentences are not logically related.
- Contradiction: There is a logical contradiction between the two sentences.

Premise: {premise}
Hypothesis: {hypothesis}
Classification:
"""

label_map = {0: "Entailment", 1: "Neutral", 2: "Contradiction"}

def preprocess_function(example):
    input_text = prefix.format(premise=example['premise'], hypothesis=example['hypothesis'])
    target_text = label_map[example['label']]

    model_inputs = tokenizer(input_text, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(target_text, max_length=20, truncation=True, padding="max_length")

    model_inputs['labels'] = labels['input_ids']

    return model_inputs

# 데이터셋 전처리 및 분할
text_data_tokenized = text_data.map(preprocess_function, remove_columns=text_data["train"].column_names)
text_data_split = text_data_tokenized['train'].train_test_split(test_size=0.2)
text_data_train, text_data_val = text_data_split['train'], text_data_split['test']
text_data_test = text_data_tokenized['validation_matched']


Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

Map:   0%|          | 0/9796 [00:00<?, ? examples/s]

KeyError: -1

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding="max_length", max_length=512)


In [ ]:
from datasets import load_metric
import numpy as np

def compute_metrics(eval_preds):
    metric = load_metric("glue", "mnli")

    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # 디코딩된 예측과 레이블을 저장할 리스트
    decoded_preds = []
    decoded_labels = []

    # 예측 결과 디코딩
    for pred in predictions:
        decoded_text = tokenizer.decode(pred, skip_special_tokens=True)
        # 'Entailment', 'Neutral', 'Contradiction' 중 하나로 매핑
        if 'entailment' in decoded_text.lower():
            decoded_preds.append(0)
        elif 'neutral' in decoded_text.lower():
            decoded_preds.append(1)
        elif 'contradiction' in decoded_text.lower():
            decoded_preds.append(2)
        else:
            # 기본값으로 'Neutral' 선택
            decoded_preds.append(1)

    # 레이블 디코딩
    for label in labels:
        decoded_text = tokenizer.decode(label, skip_special_tokens=True)
        if 'entailment' in decoded_text.lower():
            decoded_labels.append(0)
        elif 'neutral' in decoded_text.lower():
            decoded_labels.append(1)
        elif 'contradiction' in decoded_text.lower():
            decoded_labels.append(2)
        else:
            # 잘못된 레이블의 경우 'Neutral'로 처리
            decoded_labels.append(1)

    # 메트릭 계산
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)

    # 정확도 추가
    result["accuracy"] = (np.array(decoded_preds) == np.array(decoded_labels)).mean()

    return result

In [ ]:
# 학습 인자 설정
training_args = Seq2SeqTrainingArguments(
    output_dir="llama_mnli_classification",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    predict_with_generate=True,
    generation_max_length=20,
)

# Trainer 초기화
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=text_data_train,
    eval_dataset=text_data_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


In [ ]:
# 모델 학습
trainer.train()

In [ ]:
# 모델 평가 (validation 데이터 사용)
eval_results = trainer.evaluate()
print("Evaluation results on validation set:")
print(eval_results)

# 테스트 데이터에 대한 평가
test_results = trainer.evaluate(text_data_test)
print("Evaluation results on test set (validation_matched):")
print(test_results)

In [ ]:
# 예측 함수 정의
def predict(premise, hypothesis):
    input_text = prefix.format(premise=premise, hypothesis=hypothesis)
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    outputs = model.generate(**inputs, max_new_tokens=20)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("Classification:")[-1].strip()

# 예제 추론
example_premise = "The girl is playing the violin."
example_hypothesis = "The girl is holding a musical instrument."
example_prediction = predict(example_premise, example_hypothesis)
print(f"Premise: {example_premise}")
print(f"Hypothesis: {example_hypothesis}")
print(f"Prediction: {example_prediction}")